In [16]:
# ------------------------------
# AI/ML Internship Assignment - Groq API (Safe Version)
# ------------------------------

import os
import requests
import json
from getpass import getpass

# 🔒 Securely enter your API key (hidden, not saved in notebook/GitHub)
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

API_BASE = "https://api.groq.com/openai"

def call_chat_model(messages, model="llama-3.3-70b-versatile", max_tokens=256, temperature=0.2):
    url = f"{API_BASE}/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ['GROQ_API_KEY']}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature
    }

    resp = requests.post(url, headers=headers, json=payload)

    try:
        data = resp.json()
    except:
        print("❌ Could not decode response:", resp.text)
        return None

    if "choices" not in data:
        print("⚠️ API Error:", data)
        return None

    return data['choices'][0]['message']['content']


def summarize_conv(text):
    system = {"role":"system","content":"Summarize this chat in 1-2 sentences. Return plain text only."}
    user = {"role":"user","content": text}
    return call_chat_model([system,user])


def classify_conv(text):
    system = {"role":"system","content":"Classify this chat into JSON with fields {\"intent\":\"...\", \"issue\":\"...\", \"category\":\"...\"}."}
    user = {"role":"user","content": text}
    return call_chat_model([system,user])


# ------------------------------
# Tests
# ------------------------------
example1 = "User: My internet is down. Assistant: Please restart your router."
example2 = "User: I can’t log in to my account. Assistant: Please reset your password."

print("---- Summarization Example ----")
print(summarize_conv(example1))

print("\n---- Classification Example ----")
print(classify_conv(example2))

---- Summarization Example ----
The user is experiencing internet issues and the assistant suggests restarting the router as a potential solution. The conversation is about troubleshooting the user's internet problem.

---- Classification Example ----
Here is the classification of the chat in JSON format:

```
{
  "intent": "Technical Support",
  "issue": "Login Issue",
  "category": "Account Management"
}
```
